# En Github hago un Fork del repositorio y lo enlazo a VS Code

In [13]:
# Importar las bibliotecas necesarias
import pandas as pd  # Para manejar archivos CSV y bases de datos
import sqlite3  # Para interactuar con SQLite
import os  # Para comprobar la existencia de archivos

In [ ]:
import pandas as pd

# Lista de archivos CSV en el proyecto
csv_files = [
    "customers_csv.csv",
    "dim_geography.csv",
    "dim_product.csv",
    "dim_territory.csv",
    "fact_sales.csv"
]

# Recuento de filas por cada CSV
for csv_file in csv_files:
    # Cargar el CSV en un DataFrame
    df = pd.read_csv(csv_file)
    # Contar el número de registros (filas)
    num_registros = len(df)
    print(f"Archivo {csv_file} contiene {num_registros} registros.")


Archivo customers_csv.csv contiene 1360 registros.
Archivo dim_geography.csv contiene 654 registros.
Archivo dim_product.csv contiene 397 registros.
Archivo dim_territory.csv contiene 11 registros.
Archivo fact_sales.csv contiene 5954 registros.


## Comprobar tipo de datos y formato

In [17]:
import pandas as pd

# Analizar cada archivo
for file in csv_files:
    print(f"\n🔹 Análisis del archivo: {file}\n")

    # Detectar si es un CSV con delimitador `;`
    with open(file, 'r') as f:
        first_line = f.readline()
        delimiter = ';' if ';' in first_line else ','

    # Cargar CSV con el delimitador correcto
    df = pd.read_csv(file, delimiter=delimiter)

    # Crear DataFrame con nombre de columna, ejemplo de dato y tipo de dato
    analysis = pd.DataFrame({
        "Columna": df.columns,
        "Ejemplo": df.iloc[0].astype(str),  # Convertimos la primera fila a string
        "Tipo de Dato": df.dtypes.values
    })

    # Mostrar tabla en formato más limpio
    from tabulate import tabulate
    print(tabulate(analysis, headers="keys", tablefmt="pretty"))
    print("-" * 50)




🔹 Análisis del archivo: customers_csv.csv

+-------------------+-------------------+------------+--------------+
|                   |      Columna      |  Ejemplo   | Tipo de Dato |
+-------------------+-------------------+------------+--------------+
|    customer_id    |    customer_id    |   11000    |    int64     |
|   geography_id    |   geography_id    |     26     |    int64     |
|     birthdate     |     birthdate     | 08/04/1986 |    object    |
|   maritalstatus   |   maritalstatus   |     M      |    object    |
|      gender       |      gender       |     M      |    object    |
| datefirstpurchase | datefirstpurchase | 22/07/2016 |    object    |
+-------------------+-------------------+------------+--------------+
--------------------------------------------------

🔹 Análisis del archivo: dim_geography.csv

+-----------------------------+-----------------------------+-----------------+--------------+
|                             |           Columna           |     

## Crear conexión a base de datos SQLite


In [21]:

# Crear conexión a base de datos SQLite
import sqlite3
import os

# Borrar la base de datos anterior si existe (opcional pero útil porque sino hay cambios que en vez de pisar los amplia)
if os.path.exists("cycle_sales.db"):
    os.remove("cycle_sales.db")



In [22]:
#se crea base limpia

conn = sqlite3.connect("cycle_sales.db")
cursor = conn.cursor()

### CSV TABLES, tiene que ser diccionario, no vale lista

In [23]:
csv_tables = {
    "customers_csv.csv": "customers_csv",
    "dim_geography.csv": "dim_geography",
    "dim_product.csv": "dim_product",
    "dim_territory.csv": "dim_territory",
    "fact_sales.csv": "fact_sales"
}

## Procesar CSVs y guardar en la base de datos como tablas


In [ ]:
import sqlite3
import os
from csv import Sniffer


# Crear conexión a base de datos SQLite
conn = sqlite3.connect("cycle_sales.db")  # asegúrate de usar el mismo nombre si ya lo tienes
cursor = conn.cursor()


# Función para detectar automáticamente si el CSV usa coma (,) o punto y coma (;)
def detectar_separador(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        muestra = f.read(2048)  # Leer una muestra del archivo
        return Sniffer().sniff(muestra).delimiter  # Devolver el separador detectado

# Crear la conexión a la base de datos SQLite (se guarda como archivo local)
conn = sqlite3.connect("cycle_sales.db")
cursor = conn.cursor()

# Recorrer todos los archivos y sus tablas asociadas
for file_name, table_name in csv_tables.items():
    if os.path.exists(file_name):
        print(f"\n📥 Procesando: {file_name}")
        
        # Detectar el separador automáticamente
        sep = detectar_separador(file_name)
        print(f"🧪 Separador detectado para {file_name}: '{sep}'")
        
        # Leer el CSV con el separador correcto
        df = pd.read_csv(file_name, sep=sep)

        # Limpiar los nombres de columnas: minúsculas, sin espacios, sin caracteres raros
        df.columns = [col.lower().strip().replace(" ", "_") for col in df.columns]

        # Cargar los datos a la tabla correspondiente en SQLite
        df.to_sql(name=table_name, con=conn, if_exists="replace", index=False)

        # Contar registros insertados y mostrar
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        row_count = cursor.fetchone()[0]
        print(f"✅ Tabla '{table_name}' cargada con {row_count} registros")
    else:
        print(f"❌ Archivo no encontrado: {file_name}")

# Guardar cambios y cerrar conexión
conn.commit()
conn.close()

print("\n📦 Base de datos SQLite generada correctamente: cycle_sales.db")


📥 Procesando: customers_csv.csv
🧪 Separador detectado para customers_csv.csv: ';'
✅ Tabla 'customers_csv' cargada con 1360 registros

📥 Procesando: dim_geography.csv
🧪 Separador detectado para dim_geography.csv: ','
✅ Tabla 'dim_geography' cargada con 654 registros

📥 Procesando: dim_product.csv
🧪 Separador detectado para dim_product.csv: ','
✅ Tabla 'dim_product' cargada con 397 registros

📥 Procesando: dim_territory.csv
🧪 Separador detectado para dim_territory.csv: ','
✅ Tabla 'dim_territory' cargada con 11 registros

📥 Procesando: fact_sales.csv
🧪 Separador detectado para fact_sales.csv: ','
✅ Tabla 'fact_sales' cargada con 5954 registros

📦 Base de datos SQLite generada correctamente: cycle_sales.db


# Ahora hacemos un dump para generar el archivo SQLite (en la terminal)

sqlite3 cycle_sales.db .dump > cycle_sales_export.sql


### Limpieza para compatibilidad con MySQL, se genera una segunda versión de SQLite

In [ ]:
import chardet

input_file = "cycle_sales_export.sql"
output_file = "cycle_sales_export_mysql.sql"

# Detectar codificación original
with open(input_file, 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']

print(f"📄 Codificación detectada: {encoding}")

# Decodificar el archivo con la codificación correcta
content = raw_data.decode(encoding)

# Reemplazos:
content = content.replace('"', '`')  # Comillas para MySQL
content = content.replace("BEGIN TRANSACTION;", "") # Comandos incompatibles con MySQL
content = content.replace("COMMIT;", "")

# Tipos de datos más adecuados para MySQL
content = content.replace(" TEXT", " VARCHAR(255)")
content = content.replace("TEXT", "VARCHAR(255)")  # por si hay sin espacio
content = content.replace("INTEGER", "INT")
content = content.replace("REAL", "FLOAT")
content = content.replace("NUMERIC", "DECIMAL(10,2)")

# Encabezado para crear y usar la base de datos
mysql_header = """
-- Crear base de datos en MySQL
CREATE DATABASE IF NOT EXISTS sales_cycle;
USE sales_cycle;
"""

# Combinar y guardar
final_content = mysql_header.strip() + "\n\n" + content.strip()

with open(output_file, "w", encoding="utf-8") as f:
    f.write(final_content)

print("✅ Archivo SQL limpio y mejorado guardado como:", output_file)



📄 Codificación detectada: UTF-16
✅ Archivo SQL limpio y mejorado guardado como: cycle_sales_export_mysql.sql


Ahora se va a MYSQL y se abre el archivo generado